In [1]:
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/data/')

In [5]:
files = []
all_symbols = set()

In [6]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [7]:
set(['/'.join(x.split('/')[:-1]) for x in paths])

{'s3://sisyphus-general-bucket/AthenaInsights/data/csv',
 's3://sisyphus-general-bucket/AthenaInsights/data/data_prep',
 's3://sisyphus-general-bucket/AthenaInsights/data/dependent_variable',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/debug-output',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/framework',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/system',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/system/incremental/2024111907',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-54-12-710/debug-output',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts

In [8]:
[x for x in paths if not('data/csv' in x or 'data/data_prep/' in x or 'data/feature_prep' in x or 'data/dependent_variable/' in x or 'data/parquet' in x or 'data/model-artifacts/' in x)]

['s3://sisyphus-general-bucket/AthenaInsights/data/model/train/train_features.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/model/train/train_labels.parquet']

In [9]:
def filter_df(df):
    df = df.reset_index()
    df = df[df['us_eastern_timestamp']>='2024-01-01']
    df.set_index('us_eastern_timestamp')
    return df

In [12]:
new_paths = {}
for file_loc in paths:
    if 'data/csv/' in file_loc:
        new_paths[file_loc] = file_loc.replace('/data/', '/latest_data/')
        continue
        df = pd.read_csv(file_loc)
        df.to_parquet(new_paths[file_loc])
        del df

    elif 'data/data_prep/' in file_loc:
        new_paths[file_loc] = file_loc.replace('/data/', '/latest_data/')
        continue
        df = pd.read_parquet(file_loc)
        df = filter_df(df)
        df.to_parquet(new_paths[file_loc])
        del df

    elif 'data/feature_prep' in file_loc:
        new_paths[file_loc] = file_loc.replace('/data/', '/latest_data/')
        continue
        df = pd.read_parquet(file_loc)
        df = filter_df(df)
        df.to_parquet(new_paths[file_loc])
        del df

    elif 'data/dependent_variable/' in file_loc:
        new_paths[file_loc] = file_loc.replace('/data/', '/latest_data/')
        continue
        df = pd.read_parquet(file_loc)
        df = filter_df(df)
        df.to_parquet(new_paths[file_loc])
        del df

    elif 'data/parquet' in file_loc:
        new_paths[file_loc] = file_loc.replace('/data/', '/latest_data/')
        df = pd.read_parquet(file_loc)
        df = df[df.timestamp>='2024-01-01']
        df.to_parquet(new_paths[file_loc])
        del df

In [13]:
# [file_loc for file_loc in paths if 'data/parquet' in file_loc]
set(paths)-set(new_paths.keys())

{'s3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/debug-output/training_job_end.ts',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/framework/training_job_end.ts',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/system/incremental/2024111907/1732002480.algo-1.json',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/system/incremental/2024111907/1732002540.algo-1.json',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-47-20-826/profiler-output/system/training_job_end.ts',
 's3://sisyphus-general-bucket/AthenaInsights/data/model-artifacts/sagemaker-xgboost-2024-11-19-07-54-12-710/debug-output/training_job_end.ts',
 's3://sisyphus-general-bucket/AthenaInsi